In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import datetime
import pickle
from matplotlib import pyplot as plt
import plotly.express as px
import spacy

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 400)
pd.set_option("display.max_colwidth", None) # None for all of it
PARTYCOLORS =  {'AfD': '#59dcff', "CSU":"#525060", "CDU":"black", "CDU/CSU":"black", "SPD":"#e03427", "GRUENE":"#52d157", "DIE LINKE":"#a33ab5", "FDP":"#ffeb3b",}

In [2]:
stca = pd.read_feather("data/sentences_all.feather")
stca["sentence"] = stca["sentence"].str.normalize('NFC')
stca = stca.reset_index()
stca.shape

(1117882, 12)

In [3]:
stca.party.unique()

array(['CDU/CSU', 'DIE LINKE', 'SPD', 'GRUENE', 'AfD', 'CDU', 'FDP',
       'CSU'], dtype=object)

In [4]:
stca.value_counts("electoral_term")

electoral_term
19    595870
18    522012
Name: count, dtype: int64

# Selection

In [5]:
# testing
#testdf = stca[(stca["sentence"].str.contains(r"ökologisch", case=False)) & ~(stca["sentence"].str.contains(r"klima", case=False))]
#print(testdf.shape)

In [6]:
# edge cases
# enrneuerbar und fossil, umweltschutz, ökologisch

In [7]:
sentence_df1 = stca[stca["sentence"].str.contains("klima|menschengemachten? CO|1.5.{1,30}grad|2.{0,5]grad|co.?2| Erder|IPCC|Parise?r? Abkommen|Parise?r? Konferenz|energiewende|kohleausstieg|verkehrswende|treibhaus|emissionshandel", case=False)]
print(sentence_df1.shape)

(17203, 12)


In [8]:
# Exclude terms such as "gesellschaftliches Klima"
dfexclude = stca[stca["sentence"].str.contains(r"Betriebsklima|Klima für|Klima de[r|s]|politisches? Klima|gesellschaftliches? Klima|soziales? Klima", case=False)]
print(dfexclude.shape)

(176, 12)


In [9]:
ids = list(set(sentence_df1.index) - set(dfexclude.index))
climate_df = stca.loc[ids]
climate_df.shape[0]

17027

In [10]:
print("Percentage_climate: ", climate_df.shape[0] / stca.shape[0])

Percentage_climate:  0.015231482392595998


In [11]:
climate_df = climate_df.reset_index()
del climate_df["index"]
climate_df.to_feather("data/sentences_climate.feather")

# Control if additional terms lead to a bias of the green party

In [33]:
climate_only_df = stca[stca["sentence"].str.contains("klima", case=False)]
ids = list(set(climate_only_df.index) - set(dfexclude.index))
climate_only_df = stca.loc[ids]
climate_only_df.shape

(11249, 12)

In [34]:
cl_plus = climate_df[climate_df["electoral_term"] == 19]["party"].value_counts().sort_index()
cl_only = climate_only_df[climate_only_df["electoral_term"] == 19]["party"].value_counts().sort_index()
tot = stca[stca["electoral_term"] == 19]["party"].value_counts().sort_index()

Relative Percentage

In [35]:
# Including additional terms
pd.DataFrame(cl_plus/tot)[5:6].sum() / pd.DataFrame(cl_plus/tot).sum()

count    0.235731
dtype: float64

In [36]:
# Only sentences including "climate"
pd.DataFrame(cl_only/tot)[5:6].sum() / pd.DataFrame(cl_only/tot).sum()

count    0.264854
dtype: float64

Conclusion: Additional Terms do not contribute to a higher relative percentage of the Green party

# Count most common words to find relevant terms

In [110]:
from nltk.corpus import stopwords
german_stop_words = stopwords.words('german')
from HanTa import HanoverTagger as ht
from tqdm import tqdm
hannover = ht.HanoverTagger('morphmodel_ger.pgz')

In [120]:
word_count = {}
climate_only_text = " ".join(climate_only_df.sentence.to_list())

In [121]:
for word in tqdm(climate_only_text.split(" ")):
    word = word.lower().strip(" ,.:?;!–-")
    if word == "":
        continue

    if word in german_stop_words or word.startswith("klima") or len(word) < 3:
        continue

    word = str(hannover.analyze(word)[0])
    
    if word in word_count:
        word_count[word] += 1

    else:
        word_count[word] = 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229603/229603 [01:48<00:00, 2118.52it/s]


In [122]:
word_count_s = {k: v for k, v in word_count.items() if v >= 40}
word_count_s = dict(sorted(word_count_s.items(), key=lambda item: item[1], reverse=True))
with open("wordcounts_bundestag1819.json", mode="w") as f:
    f.write("\n".join([f"{key}: {value}" for key, value in word_count_s.items()]))

In [123]:
climate_only_df[climate_only_df.sentence.str.contains("pariser", case=False)]

,index,sent_id,speech_id,name,electoral_term,party,role,date,session,sentence_no,sentence_length,sentence
163870,163870,233123,8612,Matthias Miersch,18,SPD,mp,2014-12-19,77,36,17,"In diesem Sinne wünsche ich mir, dass der Klimaschutz durch die Pariser Konferenz eine neue Dynamik gewinnt."
393560,393560,562031,21181,Stefan Rebmann,18,SPD,mp,2016-09-07,186,55,33,"Wenn wir Entwicklungspolitik ernst nehmen, dann brauchen wir die finanziellen Mittel und den ernsthaften Willen für verbindliche menschenrechtliche Sorgfaltspflichten, für den Aufbau von sozialen Sicherungssystemen und für eine konsequente Umsetzung des Pariser Klimaschutzabkommens ."
524702,524702,751055,27956,Anja Weisgerber,19,CSU,mp,2017-11-21,2,41,11,Diese schrittweise Reduzierung ist übrigens auch durch das Pariser Klimaabkommen vorgezeichnet.
524788,524788,751176,27961,Lorenz Gösta Beutin,19,DIE LINKE,mp,2017-11-21,2,30,19,"Wer in unserer Gesellschaft Klimagerechtigkeit will, wer das Pariser Klimaschutzabkommen ernst nimmt, muss die Wirtschaft nach dem Menschen ausrichten."
885392,885392,1288185,52117,Lorenz Gösta Beutin,19,DIE LINKE,mp,2020-03-06,150,36,17,"Der Kohleausstieg kommt zu langsam, 2038 wird eben nicht ausreichen, wenn wir das Pariser Klimaabkommen erfüllen wollen."
885419,885419,1288232,52118,Oliver Krischer,19,GRUENE,mp,2020-03-06,150,16,28,"Was die Kohlekommission vorgeschlagen hat, wäre immer noch unzureichend gewesen, um die Ziele des Pariser Klimaabkommens zu erreichen, hätte aber die Chance geboten, diesen gesellschaftlichen Konflikt zu befrieden."
885486,885486,1288343,52125,Lisa Badum,19,GRUENE,mp,2020-03-06,150,5,21,"Damit verstoßen Sie nicht nur gegen das Pariser Klimaschutzabkommen – das ist Ihnen schon lange egal –, sondern sogar gegen Ihre eigenen Klimaziele."
787302,787302,1141457,45843,Sascha Raabe,19,SPD,mp,2019-09-11,111,40,51,"Die Europäische Union hat in dem Abkommen, das Frau Kanzlerin Merkel schon vorschnell als Riesenwurf abgefeiert hat, dem man jetzt schnell zustimmen müsste, ein Nachhaltigkeitskapitel eingearbeitet, in dem zu den Themen „Menschenrechte“, „Arbeitnehmerrechte“, „Pariser Klimaschutzabkommen“ und „Umweltschutzbestimmungen“ zwar schöne Worte stehen, wo aber bei Verstößen dagegen leider keinerlei Sanktionsmöglichkeiten genannt werden."
590912,590912,849130,32721,Anton Hofreiter,19,GRUENE,mp,2018-06-06,35,3,14,"Eine gute Möglichkeit wäre, das Pariser Klimaschutzabkommen in Zukunft in internationale Handelsverträge zu integrieren."
590913,590913,849131,32721,Anton Hofreiter,19,GRUENE,mp,2018-06-06,35,4,20,"Präsident Macron, Frankreich, hat genau das vorgeschlagen, nämlich das Pariser Klimaschutzabkommen zu einem substanziellen Bestandteil der zukünftigen Handelsverträge zu machen."
